# NNLB Baseline

In [1]:
!pip install datasets evaluate peft bitsandbytes #transformers==4.30 accelerate peft bitsandbytes
!pip install sacrebleu unbabel-comet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   

## Load

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("NilanE/ParallelFiction-Ja_En-100k", split="train")

print(raw_datasets)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

dataset-Ja_En-Massive-v2.jsonl:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/106048 [00:00<?, ? examples/s]

Dataset({
    features: ['src', 'trg', 'meta'],
    num_rows: 106048
})


In [3]:
raw_datasets.features

{'src': Value(dtype='string', id=None),
 'trg': Value(dtype='string', id=None),
 'meta': {'general': {'series_title_eng': Value(dtype='string', id=None),
   'series_title_jap': Value(dtype='string', id=None),
   'sentence_alignment_score': Value(dtype='float64', id=None)},
  'novelupdates': {'link': Value(dtype='string', id=None),
   'genres': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
   'tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
   'rating': Value(dtype='float64', id=None),
   'rating_votes': Value(dtype='int64', id=None)},
  'syosetu': {'link': Value(dtype='string', id=None),
   'series_active': Value(dtype='bool', id=None),
   'writer': Value(dtype='string', id=None),
   'fav_novel_cnt': Value(dtype='int64', id=None),
   'global_points': Value(dtype='int64', id=None)}}}

In [4]:
raw_datasets[0]

{'src': '77.素人の気づき\n「いい、アーニャ。今から行ったとしても、陛下が実際に選抜を通じて選ぶ妃は数人から数十人でしょう」\n「うん、そうだね」\n「ですが、それに既存の騎士選抜を重ね合わせることで、世の中の女達は陛下に選ばれるために、陛下が定めた基準――方向性に向かって成長していく流れになります」\n「......わあ」\n「騎士はどうしても男が中心、女が騎士になろうと考えるのは一部の物好き。ですが、玉の輿を望まない女なんてよほどの事でもなければいません。世の中の女は、陛下に気に入られる為に奮起するのです」\n「そこまで考えて......すごい!」\nアーニャは俺を尊敬しきった眼差しで見つめてきた。\n「余の考えを一瞬で読み切ったお前が凄いよ」\nそういい、微笑みながらオードリーを見た。\n「それは良いのですが、陛下は気に入った女はおられないのでしょうか?」\n「なんだ藪から棒に。この流れだと、女として、と言う意味なんだな?」\n聞きかえすと、オードリーは静かに頷いた。\n「なんでそんな事を聞く」\n「陛下は重要な事を忘れていらっしゃるように見受けられましたので」\n「重要な事?」\nなんか忘れてるか?\n「上皇陛下には多くの妃がおります。そして、\n「ふむ」\n様々な、という所で少しだけ吹き出しそうになった。\n中には臣下の妻だった女や、かつて自分の父親の妃――義理の母親だった女も妃にした。\n有名な話だ。\n時の皇帝が崩御した時は、政略的に妃にはしたが、まだ六歳という幼さ故に手付かずの女の子が一人いた。\nつまり、六歳の未亡人と言うことだ。\nそれが成長し、適齢期になった時、その美しさを見初めた父上が無理矢理自分の妻にした。\n武勇伝には事欠かないのが上皇、父上なのである。\n「臣下の妻をものにしたとき、自分の義理の母にあたる少女を手籠めにしたとき、上皇陛下は誰かに咎められまして?」\n「いいや?」\n皇帝がなぜ、その程度の事で咎められるものか。\nもっとあり得ない、非人道的な事をやっても咎められもしないのが皇帝という物だ。\n「ええ、陛下の反応そのままです」\n「何が言いたい」\n「陛下は貴族の義務と良くおっしゃってますが、貴族の権利を忘れているように思います」\n「......ふむ」\nなるほど、もっと地位と権力を享受しろって言いたいのか。\

In [5]:
raw_datasets = raw_datasets.remove_columns(["meta"])

## Preprocess

In [6]:
# Flatten and reduce the dataset
max_tok_length = 16


def flatten_examples(batch):
    flat_jp = []
    flat_en = []
    for jp, en in zip(batch["src"], batch["trg"]):
        # Too big for my resources we do a prefilter by the english size to reduce the time cost of tokenizing (as later we will be doing a filter by tokenizing length)
        i = 0
        for e, j in zip(en.split("\n"), jp.split("\n")):
            if len(e.split()) <= max_tok_length:
                flat_jp += [j]
                flat_en += [e]
                i += 1
                if i == 2:
                    ## Max of 2 sentence per character
                    break
    flat_data = {"src": flat_jp, "trg": flat_en}
    return flat_data


# Apply flattening
flat_dataset = raw_datasets.map(
    flatten_examples,
    batched=True,
    remove_columns=raw_datasets.column_names,
)

Map:   0%|          | 0/106048 [00:00<?, ? examples/s]

In [7]:
# split the data into train and test and validation
from datasets import DatasetDict

raw_datasets.shuffle()
raw_datasets = flat_dataset.train_test_split(test_size=0.2)

In [8]:
from transformers import AutoTokenizer

checkpoint = "facebook/nllb-200-distilled-600M"
# from flores200_codes import flores_codes
src_code = "jpn_Jpan"
tgt_code = "eng_Latn"
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint,
    padding=True,
    pad_to_multiple_of=8,
    src_lang=src_code,
    tgt_lang=tgt_code,
    truncation=True,
    max_length=max_tok_length,
)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [9]:
def preprocess_function(sample):
    model_inputs = tokenizer(
        sample["src"],
        text_target=sample["trg"],
    )
    return model_inputs

In [10]:
sample = raw_datasets["train"].select(range(2))
model_input = preprocess_function(sample)
print(model_input)

{'input_ids': [[256079, 151003, 255555, 248486, 248243, 249602, 250647, 253935, 249984, 8666, 31943, 14665, 7025, 250333, 2], [256079, 248059, 187846, 39843, 250532, 12723, 248300, 253415, 249604, 62324, 248237, 253131, 160691, 48672, 7265, 253935, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[256047, 69, 69669, 12646, 2790, 24973, 248075, 24725, 720, 153, 104023, 1830, 69, 248181, 11711, 1483, 248108, 2097, 2], [256047, 27666, 111, 3335, 8751, 362, 36, 248079, 117, 16978, 1537, 4157, 202, 349, 43528, 9715, 540, 158826, 796, 248075, 2]]}


In [11]:
for sample in model_input["input_ids"]:
    print(tokenizer.convert_ids_to_tokens(sample))

['jpn_Jpan', '▁「', '纏', 'う', 'は', '業', '火', '。', 'ソ', 'ード', 'エン', 'チャ', 'ント', '『', '</s>']
['jpn_Jpan', '▁', '落ち着', 'かない', '足', '取り', 'で', '玄', '関', 'に向か', 'い', '、', 'ドア', 'を開', 'ける', '。', '</s>']


In [12]:
tokenizer.batch_decode(model_input["input_ids"])

['jpn_Jpan 「纏うは業火。ソードエンチャント『</s>', 'jpn_Jpan 落ち着かない足取りで玄関に向かい、ドアを開ける。</s>']

In [13]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/169629 [00:00<?, ? examples/s]

Map:   0%|          | 0/42408 [00:00<?, ? examples/s]

In [14]:
tokenized_datasets = tokenized_datasets.filter(
    lambda x: len(x["input_ids"]) <= max_tok_length
    and len(x["labels"]) <= max_tok_length,
    desc=f"Discarding source and target sentences with more than {max_tok_length} tokens",
)

Discarding source and target sentences with more than 16 tokens:   0%|          | 0/169629 [00:00<?, ? example…

Discarding source and target sentences with more than 16 tokens:   0%|          | 0/42408 [00:00<?, ? examples…

In [15]:
dic = {}
for sample in tokenized_datasets["train"]:
    sample_length = len(sample["input_ids"])
    if sample_length not in dic:
        dic[sample_length] = 1
    else:
        dic[sample_length] += 1

for i in range(1, max_tok_length + 1):
    if i in dic:
        print(f"{i:>2} {dic[i]:>3}")

 3 2045
 4 2146
 5 4427
 6 7451
 7 9238
 8 10670
 9 11307
10 11759
11 11000
12 9827
13 8233
14 6657
15 5006
16 3583


In [16]:
for sample in tokenized_datasets["train"].select(range(5)):
    print(sample["input_ids"])
    print(sample["attention_mask"])
    print(sample["labels"])

[256079, 75627, 248871, 249672, 33724, 248318, 83822, 251006, 35960, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256047, 2820, 1465, 91, 12863, 231, 46571, 27666, 11897, 2]
[256079, 51321, 37595, 721, 82232, 253131, 35343, 249550, 170166, 19233, 253935, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256047, 11366, 349, 6471, 5884, 235615, 796, 248079, 796, 1398, 40712, 8872, 53353, 248075, 2]
[256079, 5264, 248144, 249133, 249543, 249406, 164103, 12996, 249584, 248697, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256047, 5264, 5264, 248144, 1769, 17072, 164103, 190263, 2]
[256079, 248059, 251909, 36955, 253935, 2]
[1, 1, 1, 1, 1, 1]
[256047, 3291, 248116, 248066, 3912, 888, 248075, 2]
[256079, 30592, 63271, 248833, 49789, 105323, 249519, 128681, 249746, 252252, 80737, 251823, 2]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[256047, 2366, 98, 8816, 72318, 72863, 89, 41351, 38965, 68911, 106000, 164482, 2]


In [17]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

In [18]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

## Inference Test

In [19]:
from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(
    checkpoint,
)

print(generation_config)

GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "max_length": 200,
  "pad_token_id": 1
}



In [20]:
test_batch_size = 64
batch_tokenized_test = tokenized_datasets["test"].batch(test_batch_size)

Batching examples:   0%|          | 0/25929 [00:00<?, ? examples/s]

In [22]:
number_of_batches = len(batch_tokenized_test["src"])
print(number_of_batches)
output_sequences = []
for i in range(number_of_batches):
    inputs = tokenizer(
        batch_tokenized_test["src"][i],
        max_length=max_tok_length,
        truncation=True,
        return_tensors="pt",
        padding=True,
    )
    output_batch = model.generate(
        generation_config=generation_config,
        input_ids=inputs["input_ids"].cuda(),
        attention_mask=inputs["attention_mask"].cuda(),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_code),
        max_length=max_tok_length,
        num_beams=1,
        do_sample=False,
    )
    output_sequences.extend(output_batch.cpu())

406


## Evaluation

In [24]:
from evaluate import load
from comet import download_model, load_from_checkpoint

metric = load("sacrebleu")

In [31]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace negative ids in the labels as we can't decode them.
    # labels = np.where(labels < 0, labels, tokenizer.pad_token_id)
    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j < 0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


def compute_comet(eval_preds):
    # Descargar y cargar el modelo COMET
    comet_model_path = download_model("Unbabel/wmt22-comet-da")
    comet_model = load_from_checkpoint(comet_model_path)
    preds, labels, source = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace negative ids in the labels as we can't decode them.
    # labels = np.where(labels < 0, labels, tokenizer.pad_token_id)
    for i in range(len(labels)):
        labels[i] = [tokenizer.pad_token_id if j < 0 else j for j in labels[i]]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    for i in range(len(source)):
        source[i] = [tokenizer.pad_token_id if j < 0 else j for j in source[i]]
    source = tokenizer.batch_decode(source, skip_special_tokens=True)
    source = [[s.strip()] for s in source]

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    # evaluate comet
    data = [
        {"src": s, "mt": hyp, "ref": ref}
        for hyp, ref, s in zip(decoded_preds, decoded_labels, source)
    ]
    comet_score = comet_model.predict(data, batch_size=64, gpus=1)
    return comet_score.system_score

In [27]:
metrics = compute_metrics((output_sequences, tokenized_datasets["test"]["labels"]))
print(f'BLEU score: {metrics["bleu"]}')

BLEU score: 10.5309


In [32]:
result = compute_comet(
    (
        output_sequences,
        tokenized_datasets["test"]["labels"],
        tokenized_datasets["test"]["input_ids"],
    )
)
print(result)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/f49d328952c3470eff6bb6f545d62bfdb6e66304/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 406/406 [05:33<00:00,  1.22it/s]


0.6212362829566926
